In [14]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

### Get data

In [26]:
# creat the wiki link
wiki = requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M")
soup = BeautifulSoup(wiki.text, 'lxml')
soup.prettify

# find table
table = soup.find('table',class_='wikitable')
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Harbourfront_(Toronto)" title="Harbourfront (Toronto)">Harbourfront</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Regent Park</a>
</td></tr>
<tr>
<td>M6A</td>

In [33]:
# Store parsed data into Pandas DataFrame

# iterate the wikitable to get the data
data = []
columns = []
for index, tr in enumerate(table.find_all('tr')):
    section = []
    for td in tr.find_all(['th','td']):
        section.append(td.text.rstrip())
    
# First row of data is the header
    if (index == 0):
        columns = section
    else:
        data.append(section)
        
Toronto=pd.DataFrame(data = data,columns = columns)
Toronto.head()


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### clean the data


In [34]:
#Ignore cells with a borough that is Not assigned.
Toronto = Toronto[Toronto['Borough'] != 'Not assigned']
Toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [43]:
#More than one neighborhood can exist in one postal code area. 
#Combined into one row with the neighborhoods separated with a comma

Toronto["Neighbourhood"] = Toronto.groupby("Postcode")["Neighbourhood"].transform(lambda neigh: ', '.join(neigh))

Toronto=Toronto.drop_duplicates()
Toronto.head()

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park, Harbourfront, Regen..."
6,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei..."
8,M7A,Queen's Park,Not assigned


In [46]:
# If a cell has a borough but a Not assigned neighborhood, then the neighborhood will be the same as the borough

Toronto['Neighbourhood'].replace("Not assigned", Toronto["Borough"],inplace=True)
Toronto

,Postcode,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront, Regent Park, Harbourfront, Regen..."
6,M6A,North York,"Lawrence Heights, Lawrence Manor, Lawrence Hei..."
8,M7A,Queen's Park,Queen's Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge, Malvern, Rouge, Malvern, Rouge, Malvern..."
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens, Parkview Hill, Woodbine Gard..."
17,M5B,Downtown Toronto,"Ryerson, Garden District, Ryerson, Garden Dist..."


In [47]:
Toronto.shape

(103, 3)